In [1]:
import numpy as np
import pandas as pd
import utils
from experiment.experiment_utils import set_random_seed, load_data, build_data, grid_search, makedir, save_result, load_data_multitask
from model import LogisticRegression
from pipeline.diffprep_flex_pipeline import DiffPrepFlexPipeline
from pipeline.diffprep_fix_pipeline import DiffPrepFixPipeline
import torch
import torch.nn as nn
from trainer.diffprep_trainer import DiffPrepSGD
from utils import SummaryWriter
from experiment.experiment_utils import min_max_normalize
from copy import deepcopy

/Users/siddharth/miniconda3/envs/diffprep/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_prep_pipeline(path, prep_space, params, data_dir, dataset):

    X, y = load_data_multitask(data_dir, dataset)
    X_train, y_train, X_val, y_val, X_test, y_test = build_data(X, y, random_state=params["split_seed"])

    prep_pipeline = DiffPrepFixPipeline(prep_space, temperature=params["temperature"],
                                use_sample=False,
                                diff_method=params["diff_method"],
                                init_method=params["init_method"])
    prep_pipeline.init_parameters(X_train, X_val, X_test)
    prep_pipeline.load_state_dict(torch.load(path))
    prep_pipeline.is_fitted = True
    prep_pipeline.fit(X_train)
    #prep_pipeline.eval()

    return prep_pipeline

In [3]:
class DiffPrepExperiment(object):
    """Run auto prep with one set of hyper parameters"""
    def __init__(self, data_dir, dataset, prep_space, model_name, method, fixed_pipeline_path=None):
        self.data_dir = data_dir
        self.dataset = dataset
        self.prep_space = prep_space
        self.model_name = model_name
        self.method = method
        self.fixed_pipeline_path = fixed_pipeline_path

    def run(self, params, verbose=True):        
        X, y = load_data_multitask(self.data_dir, self.dataset)
        X_train, y_train, X_val, y_val, X_test, y_test = build_data(X, y, random_state=params["split_seed"])
        
        print("Dataset shapes: ", X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

        # pre norm for diffprep flex
        if self.method == "diffprep_flex":
            X_train, X_val, X_test = min_max_normalize(X_train, X_val, X_test)
            params["patience"] = 10
            params["num_epochs"] = 3000

        # set random seed
        set_random_seed(params)
        ## transform pipeline
        # define and fit first step
        if self.fixed_pipeline_path:
            #prep_pipeline = self.fixed_pipeline
            prep_pipeline = DiffPrepFixPipeline(self.prep_space, temperature=params["temperature"],
                                use_sample=False,
                                diff_method=params["diff_method"],
                                init_method=params["init_method"])
            prep_pipeline.init_parameters(X_train, X_val, X_test)
            prep_pipeline.load_state_dict(torch.load(self.fixed_pipeline_path))
            prep_pipeline.fit(X_train)
            prep_pipeline.is_fitted = True

        elif self.method == "diffprep_fix":
            prep_pipeline = DiffPrepFixPipeline(self.prep_space, temperature=params["temperature"],
                                             use_sample=params["sample"],
                                             diff_method=params["diff_method"],
                                             init_method=params["init_method"])
            prep_pipeline.init_parameters(X_train, X_val, X_test)
        elif self.method == "diffprep_flex":
            prep_pipeline = DiffPrepFlexPipeline(self.prep_space, temperature=params["temperature"],
                            use_sample=params["sample"],
                            diff_method=params["diff_method"],
                            init_method=params["init_method"])
            prep_pipeline.init_parameters(X_train, X_val, X_test)
        else:
            raise Exception("Wrong auto prep method")

        #prep_pipeline.init_parameters(X_train, X_val, X_test)
        print("Train size: ({}, {})".format(X_train.shape[0], prep_pipeline.out_features))

        # model
        input_dim = prep_pipeline.out_features
        output_dim = len(set(y.values.ravel()))

        # model = TwoLayerNet(input_dim, output_dim)
        set_random_seed(params)
        if self.model_name == "log":
            model = LogisticRegression(input_dim, output_dim)
        else:
            raise Exception("Wrong model")

        model = model.to(params["device"])

        # loss
        loss_fn = nn.CrossEntropyLoss()

        # optimizer
        model_optimizer = torch.optim.SGD(
            model.parameters(),
            lr=params["model_lr"],
            weight_decay=params["weight_decay"],
            momentum=params["momentum"]
        )
        
        if params["prep_lr"] is None:
            prep_lr = params["model_lr"]
        else:
            prep_lr = params["prep_lr"]
    
        prep_pipeline_optimizer = None
        # torch.optim.Adam(
        #     prep_pipeline.parameters(),
        #     lr=prep_lr,
        #     betas=(0.5, 0.999),
        #     weight_decay=params["weight_decay"]
        # )

        # scheduler
        # model_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(model_optimizer, patience=patience, factor=0.1, threshold=0.001)
        prep_pipeline_scheduler = None
        model_scheduler = None

        if params["logging"]:
            logger = SummaryWriter()
        else:
            logger = None

        diff_prep = DiffPrepSGD(prep_pipeline, model, loss_fn, model_optimizer, prep_pipeline_optimizer,
                    model_scheduler, prep_pipeline_scheduler, params, writer=logger, train_pipeline=False)

        result, best_model = diff_prep.fit(X_train, y_train, X_val, y_val, X_test, y_test)
        return result, best_model, logger

In [4]:
def run_diffprep(data_dir, dataset, result_dir, prep_space, params, model_name, method, prep_pipeline_price_path):
    print("Dataset:", dataset, "Diff Method:", params["diff_method"], method)

    sample = "sample" if params["sample"] else "nosample"
    diff_prep_exp = DiffPrepExperiment(data_dir, dataset, prep_space, model_name, method, fixed_pipeline_path=prep_pipeline_price_path)
    best_result, best_model, best_logger, best_params = grid_search(diff_prep_exp, deepcopy(params))
    save_result(best_result, best_model, best_logger, best_params, result_dir, save_model=True)
    print("DiffPrep Finished. val acc:", best_result["best_val_acc"], "test acc", best_result["best_test_acc"])
    return best_result, best_model, best_logger, best_params

In [5]:
import utils
from prep_space import space
from experiment.baseline_experiment import run_baseline
import os

# define hyper parameters
params = {
    "num_epochs": 2000,
    "batch_size": 512,
    "device": "cpu",
    #"model_lr": [0.1, 0.01, 0.001],
    "model_lr": 0.01,
    "weight_decay": 0,
    "model": 'log',
    "train_seed": 1,
    "split_seed": 1,
    "method": "diffprep_fix",
    "save_model": True,
    "logging": False,
    "no_crash": False,
    "patience": 3,
    "momentum": 0.9
}

auto_prep_params = {
    "prep_lr": None,
    "temperature": 0.1,
    "grad_clip": None,
    "pipeline_update_sample_size": 512,
    "init_method": "default",
    "diff_method": "num_diff",
    "sample": False
}

DATADIR = "data"

params.update(auto_prep_params)

datasets = sorted(os.listdir(DATADIR))
dataset = "ada_prior"

print("Run {} on dataset {}".format(params["method"], dataset))

result_dir = utils.makedir(["result", params["method"], dataset])

Run diffprep_fix on dataset ada_prior


In [6]:
prep_pipeline_hpw_path = './result/diffprep_fix/ada_prior/hoursPerWeek/prep_pipeline.pth'

if params["method"] in ["diffprep_fix", "diffprep_flex"]:
    best_result, best_model, best_logger, best_params = run_diffprep(DATADIR, dataset, result_dir, space, params, params["model"], params["method"], prep_pipeline_hpw_path)
else:
    best_result, best_model, best_logger, best_params = run_baseline(DATADIR, dataset, result_dir, space, params, params["model"], params["method"], prep_pipeline_hpw_path)

Dataset: ada_prior Diff Method: num_diff diffprep_fix
Model lr 0.01
Dataset shapes:  (2738, 13) torch.Size([2738]) (912, 13) torch.Size([912]) (912, 13) torch.Size([912])
Train size: (2738, 99)


 20%|██        | 400/2000 [00:40<02:40,  9.97it/s, next_eval_time=9s, tr_loss=9.77, val_loss=20.3]  


DiffPrep Finished. val acc: 0.8212719298245614 test acc 0.8442982456140351
